In [1]:
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
import urllib
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import time

In [2]:
from urllib.parse import urlparse, parse_qs
url = urlparse("http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?serviceKey=13nWfyvckU06jS8j02cgKEde9gF7%2B9BZ%2BOFcpgpuu%2FnNRbR64QYGlNEzN7b%2F9D4RI3uFKkJ97YT9mf5X3FMIUA%3D%3D&numOfRows=10&pageNo=1&dataCd=ASOS&dateCd=HR&stnIds=108&schListCnt=10&endDt=20200101&endHh=01&startHh=01&startDt=20100101&dataType=JSON")
par_dic = parse_qs(url.query)
for key in par_dic:
    par_dic[key] = par_dic[key][0]
par_dic

{'serviceKey': '13nWfyvckU06jS8j02cgKEde9gF7+9BZ+OFcpgpuu/nNRbR64QYGlNEzN7b/9D4RI3uFKkJ97YT9mf5X3FMIUA==',
 'numOfRows': '10',
 'pageNo': '1',
 'dataCd': 'ASOS',
 'dateCd': 'HR',
 'stnIds': '108',
 'schListCnt': '10',
 'endDt': '20200101',
 'endHh': '01',
 'startHh': '01',
 'startDt': '20100101',
 'dataType': 'JSON'}

In [3]:
def get_hourly_data(start_date, start_hour,end_date,end_hour):
    start_date=start_date
    start_hour=start_hour
    end_date=end_date
    end_hour=end_hour
    url='http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'

    
    params={'serviceKey': '13nWfyvckU06jS8j02cgKEde9gF7+9BZ+OFcpgpuu/nNRbR64QYGlNEzN7b/9D4RI3uFKkJ97YT9mf5X3FMIUA==',
     'numOfRows': '10',
     'pageNo': '1',
     'dataCd': 'ASOS',
     'dateCd': 'HR',
     'stnIds': '108',
     'schListCnt': '10',
     'endDt': end_date,
     'endHh': end_hour,
     'startHh': start_hour,
     'startDt': start_date,
     'dataType': 'JSON'
           }

    resp = requests.get(url, params=params)
    #result= json.loads(resp.text)

    return resp.text

In [4]:
from urllib.parse import urlparse, parse_qs
url = urlparse("http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey=13nWfyvckU06jS8j02cgKEde9gF7%2B9BZ%2BOFcpgpuu%2FnNRbR64QYGlNEzN7b%2F9D4RI3uFKkJ97YT9mf5X3FMIUA%3D%3D&numOfRows=10&pageNo=1&dataCd=ASOS&dateCd=DAY&startDt=20100101&endDt=20100102&stnIds=108")
par_dic = parse_qs(url.query)
for key in par_dic:
    par_dic[key] = par_dic[key][0]
par_dic

{'serviceKey': '13nWfyvckU06jS8j02cgKEde9gF7+9BZ+OFcpgpuu/nNRbR64QYGlNEzN7b/9D4RI3uFKkJ97YT9mf5X3FMIUA==',
 'numOfRows': '10',
 'pageNo': '1',
 'dataCd': 'ASOS',
 'dateCd': 'DAY',
 'startDt': '20100101',
 'endDt': '20100102',
 'stnIds': '108'}

In [5]:
def get_daily_data(start_date,end_date):
    start_date=start_date
    end_date=end_date
    url='http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'

    
    params={'serviceKey': unquote('13nWfyvckU06jS8j02cgKEde9gF7%2B9BZ%2BOFcpgpuu%2FnNRbR64QYGlNEzN7b%2F9D4RI3uFKkJ97YT9mf5X3FMIUA%3D%3D'),
            'numOfRows': '200',
            'pageNo': '1',
            'dataCd': 'ASOS',
            'dateCd': 'DAY',
            'startDt': start_date,
            'endDt': end_date,
            'stnIds': '108',
            'dataType':'JSON'
           }

    resp = requests.get(url, params=params)
    result= json.loads(resp.text)
    return result

In [17]:
def data_to_DataFrame(result):
    total_data=result['response']['body']['items']['item']
    
    result = {'날짜':[],
              '최저기온':[],
              '최고기온':[],
              '비':[],
              '강수량':[],
              '날씨':[],
             }
    for data in total_data:
        avg = (data['minTa']+data['maxTa'])/2
        result['날짜'].append(data['tm'])
        result['최저기온'].append(data['minTa'])
        result['최고기온'].append(data['maxTa'])
        avg = (data['minTa']+data['maxTa'])/2
        if data['sumRn']>0:
            result['비'].append('O')
            result['날씨'].append('비')
        else:
            if avg<10:
                result['날씨'].append('추위')
            elif avg<20:
                result['날씨'].append('쌀쌀')
            else:
                result['날씨'].append('더위')
            result['비'].append('X')
        result['강수량'].append(data['sumRn'])
        
        
    df=pd.DataFrame(result)
    
    return df

In [18]:
def append_today(data):
    url = 'https://www.weatheri.co.kr/forecast/forecast01.php'
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, from_encoding='utf-8')
    attrs={'width':'615',
      'border':'0',
      'cellpadding':'1',
      'cellspacing':'1',
       'bgcolor':'#BCBFC2'
      }
    inner_table = soup.find('table',attrs=attrs)
    date = inner_table.select('td b')[0].text
    date = '2020-{0}-{1}'.format(date.split()[0][:2],date.split()[1][:2])
    high = inner_table.select('table tr td b font')[0].text
    high = high.replace('˚C','')
    low = inner_table.select('table tr td b font')[0].text
    low = low.replace('˚C','')
    sum_rain=inner_table.select('table tr td font')[-1].text
    avg = (int(low)+int(high))/2
    if sum_rain !='- mm':
        rain='O',
        sum_rain = 0
        weather = '비'
    else:
        rain='X'
        sum_rain.split()[1][:-2]
        if avg<10:
            weather = '추위'
        elif avg<20:
            weather = '쌀쌀'
        else:
            weather = '더위'
    
    new_data=pd.DataFrame({'날짜':[date],
             '최저기온':[low],
             '최고기온':[high],
             '비':[rain],
             '강수량':[sum_rain],
             '날씨':[weather]})
    
    data = data.append(new_data,ignore_index=True)
    print(data)
    return data

In [19]:
result=get_daily_data('20200501','20200605')
output=data_to_DataFrame(result)
d=append_today(output)

            날짜  최저기온  최고기온  비   강수량  날씨
0   2020-05-01  16.4  26.2  X     0  더위
1   2020-05-02    18  23.9  X     0  더위
2   2020-05-03    17  27.4  X     0  더위
3   2020-05-04  14.8  25.3  X     0  더위
4   2020-05-05  13.1  19.3  X     0  쌀쌀
5   2020-05-06  11.1  27.6  X     0  쌀쌀
6   2020-05-07  14.9  26.5  X     0  더위
7   2020-05-08  14.3  27.3  O   2.3   비
8   2020-05-09  12.7  16.8  O  24.4   비
9   2020-05-10  12.8  16.6  O   1.3   비
10  2020-05-11  12.4    22  O   3.4   비
11  2020-05-12  10.5  18.9  O   0.7   비
12  2020-05-13  10.2  21.8  X     0  쌀쌀
13  2020-05-14  12.4  25.4  X     0  쌀쌀
14  2020-05-15  14.1  20.2  O    12   비
15  2020-05-16    15  20.3  O   2.5   비
16  2020-05-17  14.6  24.9  X     0  쌀쌀
17  2020-05-18  13.2  26.8  O  29.8   비
18  2020-05-19  10.1  17.1  O    19   비
19  2020-05-20   9.2  20.1  X     0  쌀쌀
20  2020-05-21  10.9  22.9  X     0  쌀쌀
21  2020-05-22  14.8  22.7  X     0  쌀쌀
22  2020-05-23  16.6    25  X     0  더위
23  2020-05-24  14.3  20.2  O  14.5   비


In [20]:
driver = webdriver.Chrome('C:\chromedriver.exe')
driver.get('https://www.melon.com/')

In [21]:
# melon Dj 카테고리 선택
melon_dj = driver.find_element_by_xpath('//*[@id="gnb_menu"]/ul[1]/li[4]/a/span[2]').click()
song_info = {}

In [22]:
# 태그 검색

#weather = '비오는날'# 날씨 조건문 설정
def enter_weather(weather):
    weather=weather
    driver.implicitly_wait(100)
    tag_input = driver.find_element_by_class_name('input_text')
    tag_input.send_keys(weather)
    xpath = driver.find_element_by_xpath('//*[@id="conts"]/div[1]/div[2]/div[2]/button')
    driver.execute_script("arguments[0].click();", xpath)                                     # selenium에서 클릭이 안 될때 클릭 하는 법



In [23]:
# 개별 palylist에서 노래 정보 크롤링 함수
def info_select():
    body = driver.find_element_by_tag_name('tbody')
    trs = body.find_elements_by_tag_name('tr')   #노래 정보 tag


    titles = []
    singer = []
    likes = []


    for tr in trs:
        # 노래제목, 가수 각각 리스트에 저장
        name = tr.find_element_by_css_selector('div.ellipsis.rank01 a').text
        artist = tr.find_element_by_css_selector('div.ellipsis.rank02 a').text
        titles.append(name)
        singer.append(artist)

        # 좋아요 수 리스트에 저장
        cnt = tr.find_element_by_css_selector('span.cnt').text
        cnt = int(float(cnt.replace(',', '')))                   #문자열 숫자로 변환
        likes.append(cnt)
        #break

    # (노래제목, 가수) 튜플 생성
    info = list(zip(titles,singer))
    #print(info)

    # {(노래제목, 가수) : 좋아요 수 } 딕셔너리 생성
    for i in range(len(likes)):
        if info[i] in song_info:   # 같은 노래와 가수가 있으면 50점 추가
            song_info[info[i]] += 50
        else:
            song_info[info[i]] = likes[i]

In [24]:
#플레이 리스트 목록
def select_song_info():
    driver.implicitly_wait(30)
    dj_play_list = driver.find_element_by_xpath('//*[@id="djPlylstList"]/div/ul')

    lis = dj_play_list.find_elements_by_tag_name('li')
    lis_len = len(lis)

    # 각 플레이 리스트 들어갔다 나오기
    for i in range(lis_len):
        dj_play_list = driver.find_element_by_xpath('//*[@id="djPlylstList"]/div')  # 한 플레이 리스트를 들어갔다 나오면 정보가 새로 바뀌므로 오류 발생, for문 안에 lis 생성하여 저장
        lis = dj_play_list.find_elements_by_tag_name('li')
        elem = lis[i].find_element_by_css_selector('a')
        driver.implicitly_wait(30)
        driver.execute_script("arguments[0].click();", elem)  # play list click
        info_select()    # (노래제목,가수),좋아요 함수 호출
        driver.back()    # 플레이 리스트에서 목록 페이지로 돌아가기

In [26]:
result_={
        '날짜':[],
        '노래':[],
        '가수':[],
        '좋아요 수':[]
    }
for k in range(len(d['날짜'])):
    song_info = {}
    date = d['날짜'][k]
    weather = d['날씨'][k]
    print(date, weather)
#     print(weather)
    enter_weather(weather)
    select_song_info()
    song_info = sorted(song_info.items(), key=lambda x: x[1], reverse=True)

    for j in range(5):
        result_['날짜'].append(date),
        result_['노래'].append(song_info[j][0][0]),
        result_['가수'].append(song_info[j][0][1]),
        result_['좋아요 수'].append(song_info[j][1])
    driver.get('https://www.melon.com/')
    melon_dj = driver.find_element_by_xpath('//*[@id="gnb_menu"]/ul[1]/li[4]/a/span[2]').click()

2020-05-01 더위
더위
2020-05-02 더위
더위
2020-05-03 더위
더위
2020-05-04 더위
더위
2020-05-05 쌀쌀
쌀쌀
2020-05-06 쌀쌀
쌀쌀
2020-05-07 더위
더위
2020-05-08 비
비
2020-05-09 비
비
2020-05-10 비
비
2020-05-11 비
비
2020-05-12 비
비
2020-05-13 쌀쌀
쌀쌀
2020-05-14 쌀쌀
쌀쌀
2020-05-15 비
비
2020-05-16 비
비
2020-05-17 쌀쌀
쌀쌀
2020-05-18 비
비
2020-05-19 비
비
2020-05-20 쌀쌀
쌀쌀
2020-05-21 쌀쌀
쌀쌀
2020-05-22 쌀쌀
쌀쌀
2020-05-23 더위
더위
2020-05-24 비
비
2020-05-25 쌀쌀
쌀쌀
2020-05-26 비
비
2020-05-27 쌀쌀
쌀쌀
2020-05-28 쌀쌀
쌀쌀
2020-05-29 더위
더위
2020-05-30 더위
더위
2020-05-31 비
비
2020-06-01 비
비
2020-06-02 비
비
2020-06-03 더위
더위
2020-06-04 비
비
2020-06-05 더위
더위
2020-06-08 더위
더위


In [27]:
df = pd.DataFrame(result_)
df.to_excel('테스트2.xlsx')